In [ ]:
"""
CREATED FROM COMBINING RESEARCH FROM:

https://analyticsindiamag.com/a-guide-to-varma-with-auto-arima-in-time-series-modelling/
https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

and credits for how to create a VARMA model as well

https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/

"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from pmdarima import auto_arima
from sklearn import metrics

#Imported to ignore warnings from ARIMA
import warnings
warnings.filterwarnings("ignore")

#14 import statements

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#3 lines written by me

In [ ]:
"""
Easy data format and read
"""

def format_data(path):
    df = pd.read_excel(path, index_col='Year', parse_dates=True)
    df2 = df.drop(columns=['Country Name', 'Country Code'])
    df2 = df2.dropna(axis=1, how='all')
    
    return df2

In [ ]:
"""
Filling logic
"""
def fillmissing(data):
        
    newdata = data.fillna(data.mean())
    
    return newdata


In [ ]:
"""
adapted, modified and inspired from:

https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/

modified structure to fit my dataset
"""

def feature_selection(model, X_train, X_test, y_train):
    
    selector = SelectFromModel(model, max_features=3, threshold=-np.inf)
    selector.fit (X_train, y_train)
    
    select_X_train = selector.transform(X_train)
    select_X_test = selector.transform(X_test)
    
    #print (selector.get_feature_names_out())
    
    selection_model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=1, learning_rate = 0.1)
    selection_model.fit(select_X_train, y_train)
    
    select_y_pred = selection_model.predict(select_X_test)
    
    #Line 24 modified from:
    #https://stackoverflow.com/questions/54933804/how-to-get-actual-feature-names-in-xgboost-feature-importance-plot-without-retra
    selection_model.get_booster().feature_names = selector.get_feature_names_out().tolist()
    plot_importance(selection_model.get_booster())
    
    return selector ,select_X_train ,select_y_pred

In [ ]:
"""
IDEA OF USING AUTO-ARIMA IN CONJUCTION WITH VARMA GOES TO:

https://analyticsindiamag.com/a-guide-to-varma-with-auto-arima-in-time-series-modelling/
"""

def stepwisefits(data, name=""):
    stepwise_fit = auto_arima(data, trace=True, suppress_warnings=True)
    
    print(f'{name} best')
    stepwise_fit.summary()

In [ ]:
"""
line 9,10,15 (MDA) modified from: https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

line 12 (MAPE) adapted from: https://www.statology.org/mape-python/
"""

def performance_metrics(y_test, y_pred):
    
    sign1 = np.sign(np.array(y_test[1:]) - np.array(y_test[:-1]))
    sign2 = np.sign(np.array(y_pred[1:]) - np.array(y_pred[:-1]))
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test )) *100
    mae = metrics.mean_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    mda = np.mean((sign1 == sign2).astype(int))
    mean = np.mean(y_test)
    si = (rmse/mean)*100
    
    print("RMSE: ", rmse)
    print("MAPE: ", mape)
    print("MAE: ", mae)
    print("Scatter Index: ", si)
    print("MDA: ", mda)
    print("Mean of actual: ", mean)
    
#16 lines total, 9 lines me, 3 lines modified, 3 lines documentation

In [ ]:
data = format_data('/Users/farhanhabibie/Desktop/Farhan Thesis Code /UG-Project-Farhan/Multivariate More.xlsx')
data.drop(data.tail(1).index,inplace=True) #remove last row
filled_data = fillmissing(data)

In [ ]:
"""
Written by me but uses function calls to pandas and sklearn
"""

X = filled_data.drop(columns=['GDP growth (annual %)'])
y = filled_data['GDP growth (annual %)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

In [ ]:
"""
Written by me but uses function calls to xgb library
"""
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=1, learning_rate=0.2)
model.fit(X_train, y_train, verbose=False)

#2 lines from documentation

In [ ]:
selector, select_X_train, select_y_pred = feature_selection(model, X_train, X_test, y_train)

#1 line written by me

In [ ]:
best_indicators = filled_data[['GDP growth (annual %)', 
                               'General government final consumption expenditure (% of GDP)',
                              'Final consumption expenditure (% of GDP)',
                              'Foreign direct investment, net inflows (% of GDP)',
                              'Foreign direct investment, net outflows (% of GDP)',
                              'Exports of goods and services (% of GDP)', 
                               'Imports of goods and services (% of GDP)']]

#1 line written by me

In [ ]:
#Using the plot above, select columns for multivariate forecast and join with gdp growth
featurenames = selector.get_feature_names_out().tolist()
selected_data = filled_data[featurenames]
gdp_data = pd.DataFrame(y)
joined_data = pd.concat([selected_data, gdp_data], axis=1)

#5 lines written by me

In [ ]:
"""
Method for this way of checking for stationarity comes from and refactored from the guide:

https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/
"""
#Check stationarity
coint_johansen(best_indicators, -1,1).eig

#1 line from guide

In [ ]:
#Split into train test
"""
adapted and modified from: 
https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37
"""
train = best_indicators[:int(len(best_indicators)*0.8)]
test = best_indicators[int(len(best_indicators)*0.8):]

#3 lines from guide but modified

In [ ]:
for name, column in train.iteritems():
    stepwisefits(column, name=column.name)

In [ ]:
"""
Just using function calls to the VARMA library 
"""

model_V = VARMAX(train, order=(1,0))
model_Vf = model_V.fit(disp=False)
y_pred = model_Vf.predict(start=len(train), end=len(train)+len(test)-1)

In [ ]:
df_pred = pd.DataFrame(y_pred)
df2 = df_pred['GDP growth (annual %)']
test_gdp = test['GDP growth (annual %)']

In [ ]:
"""
Code written by me, however (line 9) was also used in a previous Introduction to AI Course I partook in,
at City University of London

Link to repository provided: 
https://github.com/LabiKSV/intro-to-ai-farhan-labi/blob/main/Linear%20Regression%20Label%20Encoder.ipynb
"""

df_compare = pd.DataFrame({'Actual' : test_gdp, 'Predicted' : df2})
df_compare.plot(title='GDP Growth Actual vs Predicted')
performance_metrics(test_gdp, df2)

#3 lines written by me

In [ ]:
"""86 Lines of code"""